# **Imports do Projeto**

In [8]:
import pandas as pd
import os
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# **Anos análisado**

In [9]:
anos = ['2019', '2020', '2021', '2022', '2023']

# **Pré-processamento**

In [10]:
colunas_desejadas = [
    'Data', 
    'Hora UTC', 
    'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)', 
    'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)', 
    'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)', 
    'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)', 
    'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)'
]

def processar_csv(diretorio, arquivo, diretorio_novo):
    caminho_arquivo = os.path.join(diretorio, arquivo)
    df = pd.read_csv(caminho_arquivo)

    df.columns = df.columns.str.strip()

    colunas_presentes = [coluna for coluna in colunas_desejadas if coluna in df.columns]
    df_filtrado = df[colunas_presentes]

    novo_caminho = os.path.join(diretorio_novo, arquivo)
    df_filtrado.to_csv(novo_caminho, index=False)

    print(f'Arquivo {arquivo} processado. Novo arquivo salvo como {novo_caminho}.')

for ano in anos:
    diretorio = ano
    diretorio_novo = f'{ano}-formatado'
    
    os.makedirs(diretorio_novo, exist_ok=True)

    arquivos_csv = [f for f in os.listdir(diretorio) if f.endswith('.csv')]

    for arquivo in arquivos_csv:
        processar_csv(diretorio, arquivo, diretorio_novo)

Arquivo 6-2019-CAMPOSDOSGOYTACAZES.csv processado. Novo arquivo salvo como 2019-formatado\6-2019-CAMPOSDOSGOYTACAZES.csv.
Arquivo 7-2019-ANGRADOSREIS.csv processado. Novo arquivo salvo como 2019-formatado\7-2019-ANGRADOSREIS.csv.
Arquivo 8-2019-RIODEJANEIRO.csv processado. Novo arquivo salvo como 2019-formatado\8-2019-RIODEJANEIRO.csv.
Arquivo 9-2019-VALENCA.csv processado. Novo arquivo salvo como 2019-formatado\9-2019-VALENCA.csv.
Arquivo 6-2020-CAMPOSDOSGOYTACAZES.csv processado. Novo arquivo salvo como 2020-formatado\6-2020-CAMPOSDOSGOYTACAZES.csv.
Arquivo 7-2020-ANGRADOSREIS.csv processado. Novo arquivo salvo como 2020-formatado\7-2020-ANGRADOSREIS.csv.
Arquivo 8-2020-RIODEJANEIRO.csv processado. Novo arquivo salvo como 2020-formatado\8-2020-RIODEJANEIRO.csv.
Arquivo 9-2020-VALENCA.csv processado. Novo arquivo salvo como 2020-formatado\9-2020-VALENCA.csv.
Arquivo 6-2021-CAMPOSDOSGOYTACAZES.csv processado. Novo arquivo salvo como 2021-formatado\6-2021-CAMPOSDOSGOYTACAZES.csv.
Arquiv

In [11]:
anos = ['2019-formatado', '2020-formatado', '2021-formatado', '2022-formatado', '2023-formatado']

def processar_csv(arquivo_csv, ano):
    df = pd.read_csv(arquivo_csv, sep=',') 
    
    df['Data'] = pd.to_datetime(df['Data'], format='%d/%m/%Y')
    
    colunas_para_converter = [
        'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)', 
        'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)', 
        'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)', 
        'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)', 
        'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)'
    ]
    
    for coluna in colunas_para_converter:
        df[coluna] = pd.to_numeric(df[coluna], errors='coerce')
    
    df['Semana'] = df['Data'].dt.isocalendar().week
    medias_semanal = df.groupby('Semana').agg({
        'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'mean',
        'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)': 'mean',
        'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)': 'mean',
        'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)': 'mean',
        'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)': 'mean'
    }).reset_index()
    
    nova_pasta = 'medias_semanal'
    os.makedirs(nova_pasta, exist_ok=True) 
    
    nome_arquivo_novo = os.path.join(nova_pasta, f'medias_semanal_{ano}_{os.path.basename(arquivo_csv)}')
    
    medias_semanal.to_csv(nome_arquivo_novo, index=False)
    print(f'Salvo: {nome_arquivo_novo}')

for ano in anos:
    for arquivo in os.listdir(ano):
        if arquivo.endswith('.csv'):
            caminho_arquivo = os.path.join(ano, arquivo)
            processar_csv(caminho_arquivo, ano)


Salvo: medias_semanal\medias_semanal_2019-formatado_6-2019-CAMPOSDOSGOYTACAZES.csv
Salvo: medias_semanal\medias_semanal_2019-formatado_7-2019-ANGRADOSREIS.csv
Salvo: medias_semanal\medias_semanal_2019-formatado_8-2019-RIODEJANEIRO.csv
Salvo: medias_semanal\medias_semanal_2019-formatado_9-2019-VALENCA.csv
Salvo: medias_semanal\medias_semanal_2020-formatado_6-2020-CAMPOSDOSGOYTACAZES.csv
Salvo: medias_semanal\medias_semanal_2020-formatado_7-2020-ANGRADOSREIS.csv
Salvo: medias_semanal\medias_semanal_2020-formatado_8-2020-RIODEJANEIRO.csv
Salvo: medias_semanal\medias_semanal_2020-formatado_9-2020-VALENCA.csv
Salvo: medias_semanal\medias_semanal_2021-formatado_6-2021-CAMPOSDOSGOYTACAZES.csv
Salvo: medias_semanal\medias_semanal_2021-formatado_7-2021-ANGRADOSREIS.csv
Salvo: medias_semanal\medias_semanal_2021-formatado_8-2021-RIODEJANEIRO.csv
Salvo: medias_semanal\medias_semanal_2021-formatado_9-2021-VALENCA.csv
Salvo: medias_semanal\medias_semanal_2022-formatado_6-2022-CAMPOSDOSGOYTACAZES.csv

**Porcesso para adcionar 0 casos as semanas faltantes**

In [12]:
pasta_arquivos = 'casos_dengue_rj'

semanas_padrao = list(range(1, 53))

for arquivo in os.listdir(pasta_arquivos):
    if arquivo.endswith('.csv'):
        caminho_arquivo = os.path.join(pasta_arquivos, arquivo)
        
        df = pd.read_csv(caminho_arquivo)

        if 'Semana epidemiológica dos sintomas' in df.columns and 'Casos notificados' in df.columns:
            df = df[df['Semana epidemiológica dos sintomas'] != 'Total']
            
            df['Semana epidemiológica dos sintomas'] = pd.to_numeric(df['Semana epidemiológica dos sintomas'], errors='coerce')
            df_completo = pd.DataFrame({'Semana epidemiológica dos sintomas': semanas_padrao})
            df = df_completo.merge(df, on='Semana epidemiológica dos sintomas', how='left').fillna(0)

            df['Casos notificados'] = df['Casos notificados'].astype(int)

            df.to_csv(caminho_arquivo, index=False)

print("Processamento concluído! Semanas faltantes preenchidas com 0 nos arquivos originais.")


Processamento concluído! Semanas faltantes preenchidas com 0 nos arquivos originais.


In [13]:
caminho_arquivos = 'medias_semanal/'

extremos = {
    "PRECIPITAÇÃO TOTAL, HORÁRIO (mm)": {"max": float('-inf'), "min": float('inf')},
    "TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)": {"max": float('-inf'), "min": float('inf')},
    "TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)": {"max": float('-inf'), "min": float('inf')},
    "UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)": {"max": float('-inf'), "min": float('inf')},
    "UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)": {"max": float('-inf'), "min": float('inf')}
}

for nome_arquivo in os.listdir(caminho_arquivos):
    caminho_completo = os.path.join(caminho_arquivos, nome_arquivo)
    
    if nome_arquivo.endswith('.csv'):
        df = pd.read_csv(caminho_completo)
        
        for coluna in extremos.keys():
            extremos[coluna]['max'] = max(extremos[coluna]['max'], df[coluna].max())
            extremos[coluna]['min'] = min(extremos[coluna]['min'], df[coluna].min())

df_extremos = pd.DataFrame({
    "Métrica": list(extremos.keys()),
    "Máximo": [extremos[col]["max"] for col in extremos],
    "Mínimo": [extremos[col]["min"] for col in extremos]
})

df_extremos.to_csv('extremos_dados.csv', index=False)
print("Arquivo 'extremos_dados.csv' criado com sucesso!")


Arquivo 'extremos_dados.csv' criado com sucesso!


In [14]:
input_dir = 'medias_semanal/'
output_dir = 'medias_semanal_normalizados/'
extremos_file = 'extremos_dados.csv'
os.makedirs(output_dir, exist_ok=True)

extremos_df = pd.read_csv(extremos_file)
extremos = {}

for _, row in extremos_df.iterrows():
    coluna = row['Métrica']
    extremos[coluna] = {'min': row['Mínimo'], 'max': row['Máximo']}

def normalizar(valor, min_val, max_val):
    if max_val - min_val == 0:
        return 0
    return (valor - min_val) / (max_val - min_val)

for arquivo in os.listdir(input_dir):
    if arquivo.endswith('.csv'):
        df = pd.read_csv(os.path.join(input_dir, arquivo))
        
        for coluna, limite in extremos.items():
            if coluna in df.columns:
                min_val, max_val = limite['min'], limite['max']
                df[coluna] = df[coluna].apply(lambda x: normalizar(x, min_val, max_val))
        
        df.to_csv(os.path.join(output_dir, arquivo), index=False)
        print(f'Arquivo {arquivo} normalizado e salvo em {output_dir}')


Arquivo medias_semanal_2019-formatado_6-2019-CAMPOSDOSGOYTACAZES.csv normalizado e salvo em medias_semanal_normalizados/
Arquivo medias_semanal_2019-formatado_7-2019-ANGRADOSREIS.csv normalizado e salvo em medias_semanal_normalizados/
Arquivo medias_semanal_2019-formatado_8-2019-RIODEJANEIRO.csv normalizado e salvo em medias_semanal_normalizados/
Arquivo medias_semanal_2019-formatado_9-2019-VALENCA.csv normalizado e salvo em medias_semanal_normalizados/
Arquivo medias_semanal_2020-formatado_6-2020-CAMPOSDOSGOYTACAZES.csv normalizado e salvo em medias_semanal_normalizados/
Arquivo medias_semanal_2020-formatado_7-2020-ANGRADOSREIS.csv normalizado e salvo em medias_semanal_normalizados/
Arquivo medias_semanal_2020-formatado_8-2020-RIODEJANEIRO.csv normalizado e salvo em medias_semanal_normalizados/
Arquivo medias_semanal_2020-formatado_9-2020-VALENCA.csv normalizado e salvo em medias_semanal_normalizados/
Arquivo medias_semanal_2021-formatado_6-2021-CAMPOSDOSGOYTACAZES.csv normalizado e s

In [10]:
import os
import pandas as pd

source_path = 'casos_dengue_rj/'
destination_folder = 'casos_dengue_rj_unificado/'
os.makedirs(destination_folder, exist_ok=True)

all_data = []

# Ler e unificar os dados
for file_name in os.listdir(source_path):
    if file_name.startswith("corrigido") and file_name.endswith(".csv"):
        year = file_name.split('_')[1][7:11]
        region = file_name.split('_')[-1].split('.')[0]
        file_path = os.path.join(source_path, file_name)
        df = pd.read_csv(file_path)
        df['Ano'] = year
        df['Regiao'] = region
        
        all_data.append(df)

combined_df = pd.concat(all_data, ignore_index=True)

# Salvar o arquivo unificado
output_file = os.path.join(destination_folder, 'dados_dengue_unificado.csv')
combined_df.to_csv(output_file, index=False)

print(f"Arquivo unificado criado com sucesso em: {output_file}")

# Criar arquivos separados por região
for region, group in combined_df.groupby('Regiao'):
    region_file = os.path.join(destination_folder, f'dados_dengue_{region}.csv')
    group.to_csv(region_file, index=False)
    print(f"Arquivo criado para a região '{region}' em: {region_file}")


Arquivo unificado criado com sucesso em: casos_dengue_rj_unificado/dados_dengue_unificado.csv
Arquivo criado para a região 'AR' em: casos_dengue_rj_unificado/dados_dengue_AR.csv
Arquivo criado para a região 'CG' em: casos_dengue_rj_unificado/dados_dengue_CG.csv
Arquivo criado para a região 'RJ' em: casos_dengue_rj_unificado/dados_dengue_RJ.csv
Arquivo criado para a região 'VA' em: casos_dengue_rj_unificado/dados_dengue_VA.csv


# **Análise Preditiva**